In [1]:
import pandas as pd
from utils.sp_scraper import scrape_sp500_symbols
import yfinance as yf
import pandas as pd
from tqdm import tqdm
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
import tensorflow as tf
import optuna
from optuna.integration import KerasPruningCallback
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, BatchNormalization, LeakyReLU, Dropout, Input
import time

/Users/aryanhazra/Downloads/VSCode Repos/trading_model/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Replace '.' with '-' in ticker symbols
sp_tickers = [ticker.replace(".", "-") for ticker in sorted(scrape_sp500_symbols())]
ticker_encoding = {ticker: i for i, ticker in enumerate(sp_tickers)}
data = pd.DataFrame()
# Initialize scaler dictionaries to store scalers for each ticker
scalers = {}
for ticker in tqdm(sp_tickers, desc = "Downloading data", unit="ticker"):
    # Initialize scalers
    scaler_close = StandardScaler()
    scaler_future_price = StandardScaler()

    def get_ticker_data(ticker):
        # Get max data for the ticker
        ticker_data = yf.Ticker(ticker).history(period="1y")
        return ticker_data

    try:
        # Get ticker data
        ticker_data = get_ticker_data(ticker)
    except:
        time.sleep(10)
        ticker_data = get_ticker_data(ticker)

    # Make data a column instead of index
    ticker_data.reset_index(inplace=True)

    # Make columns lowercase
    ticker_data.columns = ticker_data.columns.str.lower()

    # Add a price in 30 days column
    ticker_data['price in 30 days'] = ticker_data['close'].shift(-30)

    # Drop NA rows (last 30 days)
    ticker_data.dropna(inplace=True)

    # Replace a ticker column
    ticker_data['ticker'] = ticker_encoding[ticker]

    # Scale close column
    stock_close = ticker_data.filter(["close"])
    # Convert to numpy array
    stock_close = stock_close.values
    # Scale the data
    scaled_close = scaler_close.fit_transform(stock_close)
    # Insert scaled data into the original dataframe
    ticker_data['close'] = scaled_close

    # Scale prediction column
    stock_price_in_30_days = ticker_data.filter(["price in 30 days"])
    # Convert to numpy array
    stock_price_in_30_days = stock_price_in_30_days.values
    # Scale the data
    scaled_price_in_30_days = scaler_future_price.fit_transform(stock_price_in_30_days)
    # Insert scaled data into the original dataframe
    ticker_data['price in 30 days'] = scaled_price_in_30_days

    # Store the scalers for the ticker
    scalers[ticker] = {
        'scaler_close': scaler_close,
        'scaler_future_price': scaler_future_price
    }

    # Concat the ticker data with the main data
    data = pd.concat([data, ticker_data], ignore_index=True)

                                 Open        High         Low       Close  \
Date                                                                        
2024-04-08 00:00:00-04:00  142.737831  144.713122  141.953658  143.392960   
2024-04-09 00:00:00-04:00  144.921611  146.341041  144.425304  146.311264   
2024-04-10 00:00:00-04:00  143.353258  143.988531  142.489689  143.095184   
2024-04-11 00:00:00-04:00  144.574163  144.782619  142.608792  143.928970   
2024-04-12 00:00:00-04:00  142.529381  143.591485  139.223976  139.690506   
...                               ...         ...         ...         ...   
2025-04-01 00:00:00-04:00  116.419998  116.470001  113.360001  114.050003   
2025-04-02 00:00:00-04:00  113.120003  116.239998  112.750000  115.989998   
2025-04-03 00:00:00-04:00  113.099998  113.900002  109.570000  109.669998   
2025-04-04 00:00:00-04:00  107.139999  107.699997  102.849998  103.000000   
2025-04-07 00:00:00-04:00  102.629997  107.000000   98.510002  103.410004   

                                 Open        High         Low       Close  \
Date                                                                        
2024-04-08 00:00:00-04:00  168.235632  168.404831  167.449351  167.658356   
2024-04-09 00:00:00-04:00  167.907162  169.280681  167.558816  168.872604   
2024-04-10 00:00:00-04:00  168.006696  168.295327  166.324636  166.991486   
2024-04-11 00:00:00-04:00  167.548852  174.635401  167.369706  174.217361   
2024-04-12 00:00:00-04:00  173.441030  177.521767  173.391277  175.720276   
...                               ...         ...         ...         ...   
2025-04-01 00:00:00-04:00  219.809998  223.679993  218.899994  223.190002   
2025-04-02 00:00:00-04:00  221.320007  225.190002  221.020004  223.889999   
2025-04-03 00:00:00-04:00  205.539993  207.490005  201.250000  203.190002   
2025-04-04 00:00:00-04:00  193.889999  199.880005  187.339996  188.380005   
2025-04-07 00:00:00-04:00  177.240005  194.139999  174.619995  183.539993   

                                 Open        High         Low       Close  \
Date                                                                        
2024-04-08 00:00:00-04:00  163.262741  165.925729  162.481198  163.832001   
2024-04-09 00:00:00-04:00  163.764456  164.555643  161.516348  164.160049   
2024-04-10 00:00:00-04:00  162.664513  163.986371  161.477748  163.089050   
2024-04-11 00:00:00-04:00  163.253056  163.417079  161.352295  161.632111   
2024-04-12 00:00:00-04:00  160.151841  160.151841  155.818151  158.038559   
...                               ...         ...         ...         ...   
2025-04-01 00:00:00-04:00  210.520004  212.149994  205.600006  206.270004   
2025-04-02 00:00:00-04:00  207.220001  207.220001  200.339996  205.190002   
2025-04-03 00:00:00-04:00  210.300003  210.690002  201.240005  201.639999   
2025-04-04 00:00:00-04:00  200.000000  200.000000  186.169998  186.960007   
2025-04-07 00:00:00-04:00  181.029999  188.175003  176.309998  182.414993   

                                 Open        High         Low       Close  \
Date                                                                        
2024-04-08 00:00:00-04:00  160.360001  161.179993  159.289993  160.270004   
2024-04-09 00:00:00-04:00  161.550003  163.205002  159.910004  162.660004   
2024-04-10 00:00:00-04:00  159.630005  160.960007  158.630005  160.279999   
2024-04-11 00:00:00-04:00  159.000000  166.220001  158.899994  165.419998   
2024-04-12 00:00:00-04:00  162.419998  162.712006  158.779999  159.369995   
...                               ...         ...         ...         ...   
2025-04-01 00:00:00-04:00  119.669998  121.349998  117.800003  121.180000   
2025-04-02 00:00:00-04:00  119.480003  124.589996  119.080002  122.800003   
2025-04-03 00:00:00-04:00  116.620003  117.180000  111.800003  113.970001   
2025-04-04 00:00:00-04:00  109.260002  110.419998  104.730003  106.660004   
2025-04-07 00:00:00-04:00  103.010002  110.910004   99.889999  105.405602   

                                 Open        High         Low       Close  \
Date                                                                        
2024-04-08 00:00:00-04:00  109.077103  109.253509  107.940268  108.312675   
2024-04-09 00:00:00-04:00  108.381286  110.321745  108.371491  110.194344   
2024-04-10 00:00:00-04:00  108.998696  109.439706  108.146069  108.949692   
2024-04-11 00:00:00-04:00  109.420124  109.743535  108.106882  109.224113   
2024-04-12 00:00:00-04:00  108.603625  108.830153  106.850525  107.461159   
...                               ...         ...         ...         ...   
2025-04-01 00:00:00-04:00  132.479996  132.630005  131.080002  131.690002   
2025-04-02 00:00:00-04:00  130.639999  132.130005  129.759995  131.929993   
2025-04-03 00:00:00-04:00  131.910004  133.639999  131.479996  131.630005   
2025-04-04 00:00:00-04:00  131.229996  132.399994  124.000000  124.440002   
2025-04-07 00:00:00-04:00  122.290001  126.500000  120.470001  123.050003   

...

                                 Open        High         Low       Close  \
Date                                                                        
2024-04-08 00:00:00-04:00  114.399822  114.843611  113.206515  114.458992   
2024-04-09 00:00:00-04:00  114.735113  115.533945  110.109816  110.326782   
2024-04-10 00:00:00-04:00  109.468789  113.926433  108.482583  112.644371   
2024-04-11 00:00:00-04:00  111.460922  112.752856  110.691683  111.460922   
2024-04-12 00:00:00-04:00  110.455001  111.125621  107.782385  108.610794   
...                               ...         ...         ...         ...   
2025-04-01 00:00:00-04:00  136.610001  138.570007  133.960007  138.369995   
2025-04-02 00:00:00-04:00  135.169998  142.779999  135.169998  141.520004   
2025-04-03 00:00:00-04:00  128.529999  128.899994  122.139999  123.449997   
2025-04-04 00:00:00-04:00  116.430000  117.180000  105.610001  108.680000   
2025-04-07 00:00:00-04:00  103.309998  119.489998  102.580002  109.934998   

                                Open       High        Low      Close  \
Date                                                                    
2024-04-08 00:00:00-04:00  75.199997  77.220001  75.169998  76.070000   
2024-04-09 00:00:00-04:00  76.510002  78.989998  76.279999  78.930000   
2024-04-10 00:00:00-04:00  77.000000  77.389999  74.279999  74.949997   
2024-04-11 00:00:00-04:00  75.199997  75.970001  73.849998  75.769997   
2024-04-12 00:00:00-04:00  75.099998  75.150002  73.389999  74.190002   
...                              ...        ...        ...        ...   
2025-04-01 00:00:00-04:00  59.560001  60.000000  58.180000  59.150002   
2025-04-02 00:00:00-04:00  58.740002  61.060001  58.270000  60.779999   
2025-04-03 00:00:00-04:00  58.580002  59.299999  55.200001  55.389999   
2025-04-04 00:00:00-04:00  53.070000  53.770000  50.099998  53.000000   
2025-04-07 00:00:00-04:00  51.080002  55.040001  50.689999  52.584999   

                            Volume  Dividends  Sto

                                 Open        High         Low       Close  \
Date                                                                        
2024-04-08 00:00:00-04:00  119.436021  120.768377  119.293268  120.587555   
2024-04-09 00:00:00-04:00  121.101464  123.537766  121.082434  123.337921   
2024-04-10 00:00:00-04:00  118.550964  118.684199  115.838673  116.828423   
2024-04-11 00:00:00-04:00  117.846720  118.674679  115.990937  117.913338   
2024-04-12 00:00:00-04:00  117.627835  117.646872  115.381866  115.952873   
...                               ...         ...         ...         ...   
2025-04-01 00:00:00-04:00   93.279999   93.540001   90.070000   91.150002   
2025-04-02 00:00:00-04:00   90.849998   92.870003   90.089996   92.330002   
2025-04-03 00:00:00-04:00   90.760002   91.470001   86.000000   86.250000   
2025-04-04 00:00:00-04:00   84.500000   84.790001   81.199997   81.279999   
2025-04-07 00:00:00-04:00   79.019997   82.709999   76.542503   79.695000   

                                 Open        High         Low       Close  \
Date                                                                        
2024-04-08 00:00:00-04:00  113.535524  114.081741  113.096594  113.467247   
2024-04-09 00:00:00-04:00  113.876901  114.374353  113.350189  113.769608   
2024-04-10 00:00:00-04:00  112.052926  112.404067  110.911720  112.248009   
2024-04-11 00:00:00-04:00  113.057586  113.057586  111.126308  111.565231   
2024-04-12 00:00:00-04:00  111.399417  112.647916  110.443538  111.184830   
...                               ...         ...         ...         ...   
2025-04-01 00:00:00-04:00  154.429993  155.270004  152.990005  154.940002   
2025-04-02 00:00:00-04:00  154.750000  155.619995  153.600006  155.210007   
2025-04-03 00:00:00-04:00  155.589996  157.619995  155.070007  155.289993   
2025-04-04 00:00:00-04:00  154.479996  156.000000  147.070007  147.809998   
2025-04-07 00:00:00-04:00  144.580002  149.110001  141.589996  146.160004   

                                 Open        High         Low       Close  \
Date                                                                        
2024-04-08 00:00:00-04:00  176.964225  180.858236  176.092423  180.344833   
2024-04-09 00:00:00-04:00  180.993843  183.028017  180.993843  182.940842   
2024-04-10 00:00:00-04:00  178.494702  180.577312  177.603538  180.267349   
2024-04-11 00:00:00-04:00  181.390995  181.846266  178.087881  180.122055   
2024-04-12 00:00:00-04:00  179.569909  180.034861  177.002956  178.020050   
...                               ...         ...         ...         ...   
2025-04-01 00:00:00-04:00  215.570007  215.919998  211.429993  214.509995   
2025-04-02 00:00:00-04:00  214.539993  216.470001  212.419998  215.610001   
2025-04-03 00:00:00-04:00  212.789993  215.539993  205.179993  205.860001   
2025-04-04 00:00:00-04:00  203.190002  205.509995  191.490005  191.699997   
2025-04-07 00:00:00-04:00  189.960007  195.389999  182.559998  188.544998   

In [ ]:
# Prepare the data for the model

# Initialize scalers
scaler_ticker = StandardScaler()

# Scale the ticker column
stock_ticker = data.filter(["ticker"])
stock_ticker = stock_ticker.values
scaled_ticker = scaler_ticker.fit_transform(stock_ticker)
data['ticker'] = scaled_ticker

# Group the data by ticker
grouped_dfs = data.groupby('ticker')


In [ ]:
# Create a sliding window for our stock (60 days in past to predict 30 days in future)
x_train, y_train = [], []
for ticker, df in tqdm(grouped_dfs, desc= "Creating sliding windows", unit="ticker"):
    # Sort df by date
    df.sort_values(by='date', inplace=True)

    # Loop through the DataFrame to create sliding windows
    for i in range(60, len(df) - 30):

        # Get the past 60 close prices
        close_prices = df.iloc[i - 60:i]['close'].values.reshape(-1, 1)  # shape (60, 1)

        # Repeat the ticker value for each timestep
        ticker_feature = np.full((60, 1), ticker)  # shape (60, 1)

        # Combine features: shape will be (60, 2)
        features = np.hstack((close_prices, ticker_feature))

        # Append the full (60, 2) array to x_train
        x_train.append(features)

        # Append the 'price in 30 days' value at the 60th row to y_train
        y_train.append(df.iloc[i]['price in 30 days'])

# Convert x_train and y_train to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)

# Reshape x_train to be 3D for LSTM input
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], x_train.shape[2]))


Creating sliding windows: 100%|██████████| 503/503 [00:03<00:00, 140.14ticker/s]


In [ ]:
# Build the Model
model = keras.models.Sequential()

model.add(Bidirectional(LSTM(64, return_sequences=True), input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(Dropout(0.3))

model.add(Bidirectional(LSTM(64, return_sequences=False)))
model.add(Dropout(0.3))

model.add(Dense(128, kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.3))

model.add(Dense(64, kernel_regularizer=regularizers.l2(0.001)))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.3))

model.add(Dense(1))

model.summary()
model.compile(optimizer="adam",
              loss="mae",
              metrics=[keras.metrics.RootMeanSquaredError()])


early_stopping = EarlyStopping(
    monitor='val_loss',        # You can also use 'val_root_mean_squared_error' if you're tracking that
    patience=5,                # Wait for 5 epochs with no improvement
    restore_best_weights=True # Revert to the best weights after stopping
)

training = model.fit(
    x_train, y_train,
    epochs=200,                # Max number of epochs
    batch_size=8,
    validation_split=0.1,      # Use part of training data for validation
    callbacks=[early_stopping]
)
model.save("model1.keras")

2025-04-07 00:21:00.056478: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2025-04-07 00:21:00.056546: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-04-07 00:21:00.056567: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
I0000 00:00:1743999660.056970  148051 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1743999660.057173  148051 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/Users/aryanhazra/Downloads/VSCode Repos/trading_model/.venv/lib/python3.10/site-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)   │ (None, 60, 128)        │        34,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 158,465 (619.00 KB)

 Trainable params: 158,209 (618.00 KB)

 Non-trainable params: 256 (1.00 KB)

Epoch 1/200


2025-04-07 00:21:00.783959: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


7406/7406 ━━━━━━━━━━━━━━━━━━━━ 365s 49ms/step - loss: 0.7568 - root_mean_squared_error: 0.8119 - val_loss: 0.6326 - val_root_mean_squared_error: 0.7992
Epoch 2/200
7406/7406 ━━━━━━━━━━━━━━━━━━━━ 361s 49ms/step - loss: 0.6096 - root_mean_squared_error: 0.7646 - val_loss: 0.6320 - val_root_mean_squared_error: 0.7896
Epoch 3/200
7406/7406 ━━━━━━━━━━━━━━━━━━━━ 360s 49ms/step - loss: 0.5945 - root_mean_squared_error: 0.7503 - val_loss: 0.6243 - val_root_mean_squared_error: 0.7966
Epoch 4/200
7406/7406 ━━━━━━━━━━━━━━━━━━━━ 360s 49ms/step - loss: 0.5783 - root_mean_squared_error: 0.7321 - val_loss: 0.6011 - val_root_mean_squared_error: 0.7597
Epoch 5/200
7406/7406 ━━━━━━━━━━━━━━━━━━━━ 361s 49ms/step - loss: 0.5632 - root_mean_squared_error: 0.7168 - val_loss: 0.6090 - val_root_mean_squared_error: 0.7801
Epoch 6/200
7406/7406 ━━━━━━━━━━━━━━━━━━━━ 361s 49ms/step - loss: 0.5479 - root_mean_squared_error: 0.7026 - val_loss: 0.6273 - val_root_mean_squared_error: 0.7962
Epoch 7/200
7406/7406 ━━━━━━